<a href="https://colab.research.google.com/github/joaodpcm/MDA/blob/preprocessing_data_david/preprocessing_data_mda_david.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Exploration**

---






# Importing Packages


In [4]:
# Importing packages 
import pandas as pd
import numpy as np
import seaborn as sn
import os
!pip install pandas-profiling
!pip install ydata-profiling
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Clone repository

In [5]:
# Repository path
repo_path = 'https://github.com/joaodpcm/MDAgit'
#Accessing GitHub repository
!git clone <https://github.com/joaodpcm/MDAgit>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `git clone <https://github.com/joaodpcm/MDAgit>'


# Loading data

In [6]:
# Setting google drive access
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
# Setting package for drive
!pip install pyDrive
from google.colab import files
# Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')
# Set path to the directory containing the meta dataset
path_copy_meta = "/content/drive/MyDrive/modern_data_analysis/data/meteo_data/01_Metadata_v2.csv"
df_meta = pd.read_csv(path_copy_meta)
# Access to the year's quarter dataset 
path_copy_1 = "/content/drive/MyDrive/modern_data_analysis/data/meteo_data/LC_2022Q3.csv"
path_copy_2 = "/content/drive/MyDrive/modern_data_analysis/data/meteo_data/LC_2022Q3.csv"
path_copy_3 = "/content/drive/MyDrive/modern_data_analysis/data/meteo_data/LC_2022Q3.csv"
path_copy_4 = "/content/drive/MyDrive/modern_data_analysis/data/meteo_data/LC_2022Q3.csv"
# Loading each year's quarter dataset
df_lc1 = pd.read_csv(path_copy_1)
df_lc2 = pd.read_csv(path_copy_2)
df_lc3 = pd.read_csv(path_copy_3)
df_lc4 = pd.read_csv(path_copy_4)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Merging locations datasets

The original datasets of the locations are concatenated for reaching more information about the complete year. Information about the quarter of the year is kept. 

In [8]:
# Add a column to identify the quarter
df_lc1['quarter'] = 'Q1'
df_lc2['quarter'] = 'Q2'
df_lc3['quarter'] = 'Q3'
df_lc4['quarter'] = 'Q4'

# Concatenate the four datasets into a single dataframe
df_lc = pd.concat([df_lc1, df_lc2, df_lc3, df_lc4], ignore_index=True)

# Print the final dataframe
print(df_lc.head())

               DATEUTC      ID  LC_HUMIDITY  LC_DWPTEMP  LC_n  LC_RAD  \
0  2022-07-01 00:10:00  LC-002         99.0       11.39  38.0     0.0   
1  2022-07-01 00:20:00  LC-002         99.0       11.30  37.0     0.0   
2  2022-07-01 00:30:00  LC-002         99.0       11.08  38.0     0.0   
3  2022-07-01 00:40:00  LC-002         99.0       10.78  37.0     0.0   
4  2022-07-01 00:50:00  LC-002         99.0       10.44  38.0     0.0   

   LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED  ... Month  Day  Hour  \
0        0.0           0.0         0.0           0.0  ...     7    1     0   
1        0.0           0.0         0.0           0.0  ...     7    1     0   
2        0.0           0.0         0.0           0.0  ...     7    1     0   
3        0.0           0.0         0.0           0.0  ...     7    1     0   
4        0.0           0.0         0.0           0.0  ...     7    1     0   

   Minute  LC_RAD60  LC_TEMP_QCL0  LC_TEMP_QCL1  LC_TEMP_QCL2  LC_TEMP_QCL3  \
0      10    

In [9]:
print(df_meta.head())

       ID     LAT    LON  ALT  LCZ  LC  building_height       SVF  \
0  LC-002  50.847  4.756   47    9   1         4.280000  0.610367   
1  LC-003  50.870  4.728   44    5   1         5.038286  0.691315   
2  LC-004  50.871  4.685   31    9   1         9.050964  0.798586   
3  LC-005  50.881  4.713   26    2   1         5.050000  0.685410   
4  LC-006  50.912  4.716   13    6   1         0.000000  0.954032   

   installation_height  
0                    2  
1                    2  
2                    2  
3                    2  
4                    2  


## Merging metadata with locations dataset

The information about the included features within the metadata is relevant for the main goal we established. For that reason the final locations dataset and the metadata will be concadenated.

In [10]:
# merging

# merge the two datasets by ID
df_meteo = pd.merge(df_meta, df_lc, on='ID')

# display the merged dataset
print(df_meteo.head())

       ID     LAT    LON  ALT  LCZ  LC  building_height       SVF  \
0  LC-002  50.847  4.756   47    9   1             4.28  0.610367   
1  LC-002  50.847  4.756   47    9   1             4.28  0.610367   
2  LC-002  50.847  4.756   47    9   1             4.28  0.610367   
3  LC-002  50.847  4.756   47    9   1             4.28  0.610367   
4  LC-002  50.847  4.756   47    9   1             4.28  0.610367   

   installation_height              DATEUTC  ...  Month  Day  Hour  Minute  \
0                    2  2022-07-01 00:10:00  ...      7    1     0      10   
1                    2  2022-07-01 00:20:00  ...      7    1     0      20   
2                    2  2022-07-01 00:30:00  ...      7    1     0      30   
3                    2  2022-07-01 00:40:00  ...      7    1     0      40   
4                    2  2022-07-01 00:50:00  ...      7    1     0      50   

   LC_RAD60  LC_TEMP_QCL0  LC_TEMP_QCL1  LC_TEMP_QCL2 LC_TEMP_QCL3  quarter  
0       0.0         11.50         11.5

#### Exporting meteo data

In [11]:
# Exporting pre-cleaned meteo dataset

# df_meteo.to_csv("merged_data.csv", index=False)

# Exploratory Data Analysis

## Using profiling

In [12]:
# Using profiling 
# Profile report
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
profile1 = ProfileReport(
    df_meteo, title="Exploratory Data Analysis", html={"style": {"full_width": True}}, sort=None
)

In [ ]:
# Html file to report
profile1.to_notebook_iframe()
profile1.to_file("EDA.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## Classical Data Exploration

### Exploring missingness


In [ ]:
# Defining function for missing values within features
def missing_values(df):
    """
    A function to identify missing values in a pandas DataFrame.
    """
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_values = pd.concat([missing, missing_pct], axis=1, keys=['Total Missing', 'Percentage'])
    missing_values.sort_values(by='Total Missing', ascending=False, inplace=True)
    return missing_values

In [ ]:
# Identifying missing values in features 
missing_values(df_meta)

In [ ]:
# Defining function for identifying missing values in rows 
def get_rows_with_missing_values(df):
    """
    A function to get the rows with missing values in a pandas DataFrame.
    """
    rows_with_missing = df[df.isnull().any(axis=1)]
    return rows_with_missing

In [ ]:
# # Identifying missing values in rows 
# rows_with_missing(df_meta)

In [ ]:
# # Binding zipcode, lattitude and longitude
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="zipcode2", timeout=10)
# def get_zipcode2(row):
#     if pd.notnull(row['property_zipcode']):
#         return row['property_zipcode'].replace(" ", "") #delete spaces
#     else:
#         lat = row['property_lat']
#         lon = row['property_lon']
#         if pd.notnull(lat) and pd.notnull(lon):
#             location = geolocator.reverse(f"{lat},{lon}", exactly_one=True)
#             address = location.raw['address']
#             return address.get('postcode')
#         else:
#             return None
# df['zipcode2'] = df.apply(get_zipcode2, axis=1)

# Feature engineering


# New Section